<a href="https://colab.research.google.com/github/abiflynn/data_engineering/blob/main/web_scraping/wikipedia_web_scraping_table_page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
!pip install --upgrade beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


In [2]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
#copy the url . find url and store it in a variable
url="https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"

In [4]:
# download html with a get request
headers = {'Accept-Language': 'en-US,en;q=0.8'}
response = requests.get(url, headers = headers)
response.status_code

200

In [6]:
# parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [7]:
soup.select("td a")[0].get("title") # city name


'Istanbul'

In [8]:
num_iter = len(soup.select("td a"))
num_iter

234

In [9]:
for i in range(num_iter):
    print(soup.select("td a")[i].get_text())

Istanbul
[a]
Turkey
[b]

41°00′49″N 28°57′18″E﻿ / ﻿41.013611°N 28.955°E﻿ / 41.013611; 28.955﻿ (1 Istanbul)
[2]
Moscow
[c]
Russia

55°45′00″N 37°37′00″E﻿ / ﻿55.75°N 37.616667°E﻿ / 55.75; 37.616667﻿ (2 Moscow)
[3]
London
United Kingdom

51°30′26″N 0°07′39″W﻿ / ﻿51.507222°N 0.1275°W﻿ / 51.507222; -0.1275﻿ (3 London)
[4]
Saint Petersburg
Russia

59°57′N 30°18′E﻿ / ﻿59.95°N 30.3°E﻿ / 59.95; 30.3﻿ (4 Saint Petersburg)
[3]
Berlin
Germany

52°31′00″N 13°23′00″E﻿ / ﻿52.516667°N 13.383333°E﻿ / 52.516667; 13.383333﻿ (5 Berlin)
[5]
Madrid
Spain

40°23′00″N 3°43′00″W﻿ / ﻿40.383333°N 3.716667°W﻿ / 40.383333; -3.716667﻿ (6 Madrid)
[6]
Kyiv
Ukraine

50°27′00″N 30°31′24″E﻿ / ﻿50.45°N 30.523333°E﻿ / 50.45; 30.523333﻿ (7 Kiev)
[7]
Rome
Italy

41°54′N 12°30′E﻿ / ﻿41.9°N 12.5°E﻿ / 41.9; 12.5﻿ (8 Rome)
[8]
Baku
[d]
Azerbaijan

40°24′N 49°54′E﻿ / ﻿40.4°N 49.9°E﻿ / 40.4; 49.9﻿ (Baku)
[9]
Bucharest
Romania

44°25′57″N 26°06′14″E﻿ / ﻿44.4325°N 26.103889°E﻿ / 44.4325; 26.103889﻿ (9 Bucharest)
[10]
Paris
France



In [11]:
#soup.select("table")

In [12]:
soup.select("td a")[4].get_text()

''

In [14]:
# created a dataframe of european cities 
table =soup.find(lambda tag: tag.name=='table')# find the table 
rows = table.findAll(lambda tag: tag.name=='a' and tag.has_attr("title"))# find rows wth tag a in table
# rows contains links of cities and corresponding countries such that links of cities in odd and links of countries
# in even positions respectively
#type(rows)#.text
# create a empty list city
cities=[] 
# append the cities stored in odd positions
for i in range(0,len(rows),2):
    cities.append(rows[i].text)
    
# create a empty list country   
countries=[]
# append the countries stored in odd positions
for i in range(1,len(rows),2):
    countries.append(rows[i].text)

#span tag contains population and european population(mostly blanks)
#european poulation has parent as td with attribute align

#table =soup.find(lambda tag: tag.name=='table')# find the table 
rows = table.findAll(lambda tag: tag.name=='span' and tag.has_attr("data-sort-value")and not tag.has_attr("style") and not tag.parent.has_attr("align"))
population=[] 
# append the cities stored in odd positions
for i in range(0,len(rows),1):
    population.append(rows[i].text)
   
#table =soup.find(lambda tag: tag.name=='table')# find the table 
long = table.findAll('span' ,class_="longitude")
lat=table.findAll('span' ,class_="latitude")

euro_cities_df = pd.DataFrame(list(zip(cities,countries,population,long,lat)),columns=["city","country","population","longititude","latitude"])

euro_cities_df.head() 

,city,country,population,longititude,latitude
0,Istanbul,Turkey,"15,907,951",[28°57′18″E],[41°00′49″N]
1,Moscow,Russia,"13,097,539",[37°37′00″E],[55°45′00″N]
2,London,United Kingdom,"8,799,800",[0°07′39″W],[51°30′26″N]
3,Saint Petersburg,Russia,"8,173,941",[30°18′E],[59°57′N]
4,Berlin,Germany,"5,598,486",[13°23′00″E],[52°31′00″N]
